In [202]:
import pandas as pd 
import numpy as np

from sklearn import metrics
import seaborn as sn
import confusion_matrix_pretty_print
from confusion_matrix_pretty_print import plot_confusion_matrix_from_data
from sklearn import tree
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix,classification_report,precision_score

In [164]:
data = pd.read_csv('data1000v1.csv')

In [165]:
data.shape

(1000, 1139)

In [166]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,1129,1130,1131,1132,1133,1134,1135,1136,1137,1138
0,DB00384,DB00248,0,0.001651,0.001783,0.001572,0.001935,0.001869,0.001997,0.001921,...,0.001518,0.001730,0.001177,0.001660,0.001451,0.001067,0.001527,0.001565,0.001595,0.001592
1,DB00248,DB00384,0,0.001519,0.001741,0.001426,0.001485,0.001971,0.001518,0.001526,...,0.001483,0.001508,0.001787,0.002060,0.001366,0.001041,0.001414,0.001544,0.001435,0.001480
2,DB00244,DB04835,0,0.001666,0.001544,0.001413,0.001586,0.001652,0.001541,0.001570,...,0.001548,0.001827,0.001109,0.001649,0.001583,0.001372,0.001742,0.001527,0.001811,0.001697
3,DB04835,DB00244,0,0.001572,0.001523,0.001554,0.001496,0.001511,0.001398,0.001510,...,0.001500,0.001568,0.001213,0.001553,0.001437,0.001074,0.001546,0.001854,0.001706,0.001742
4,DB00278,DB00438,0,0.001600,0.001588,0.001563,0.001519,0.001551,0.001390,0.001558,...,0.001554,0.001432,0.001036,0.001328,0.001923,0.001469,0.001791,0.001493,0.001456,0.001638


In [167]:
X_train, X_test = data.values[:800,3:], data.values[800:,3:]
y_train, y_test = data.values[:800,2], data.values[800:,2]

In [168]:
X_train[0]

array([0.0016507470000000002, 0.0017831, 0.0015716870000000002, ...,
       0.001564923, 0.0015947000000000001, 0.00159174], dtype=object)

In [169]:
1136/2/2/2/2

71.0

In [170]:
16*71

1136

In [171]:
#reshape data to fit model
X_train = X_train.reshape(800,16,71,1)
X_test = X_test.reshape(200,16,71,1)

In [172]:
X_train.shape

(800, 16, 71, 1)

In [173]:
y_train = y_train + 1
y_test = y_test + 1
print(y_train[0], y_test[0])

1 1


In [174]:
from keras.utils import to_categorical
#one-hot encode target column
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_test[0]

array([0., 1., 0.], dtype=float32)

In [248]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=2, activation='relu', padding='same', input_shape=(16,71,1)))
model.add(Conv2D(32, kernel_size=2, activation='relu', padding='same'))
model.add(Conv2D(32, kernel_size=2, activation='relu', padding='same'))
model.add(Conv2D(32, kernel_size=2, activation='relu', padding='same'))
model.add(Conv2D(32, kernel_size=2, activation='relu', padding='same'))
model.add(Conv2D(32, kernel_size=2, activation='relu', padding='same'))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

In [249]:
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50)

Train on 800 samples, validate on 200 samples
Epoch 1/50
800/800 [==============================] - 11s 13ms/step - loss: 0.6127 - acc: 0.8325 - val_loss: 0.2667 - val_acc: 0.9400
Epoch 2/50
800/800 [==============================] - 10s 12ms/step - loss: 0.5028 - acc: 0.8550 - val_loss: 0.2935 - val_acc: 0.9400
Epoch 3/50
800/800 [==============================] - 10s 12ms/step - loss: 0.4879 - acc: 0.8550 - val_loss: 0.2808 - val_acc: 0.9400
Epoch 4/50
800/800 [==============================] - 10s 12ms/step - loss: 0.4667 - acc: 0.8550 - val_loss: 0.3353 - val_acc: 0.9400
Epoch 5/50
800/800 [==============================] - 10s 12ms/step - loss: 0.3139 - acc: 0.8712 - val_loss: 0.3350 - val_acc: 0.9250
Epoch 6/50
800/800 [==============================] - 10s 13ms/step - loss: 0.1555 - acc: 0.9438 - val_loss: 0.5741 - val_acc: 0.8450
Epoch 7/50
800/800 [==============================] - 10s 13ms/step - loss: 0.0847 - acc: 0.9700 - val_loss: 0.6434 - val_acc: 0.8650
Epoch 8/50
800/8

In [ ]:
#predict first 4 images in the test set
predit = model.predict(X_test)
predit[:4]

In [ ]:
#actual results for first 4 images in test set
y_test[:4]

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.metrics import auc
prec, rec, thr = precision_recall_curve(y_test[:,2],predit[:,2])
aupr_val = auc(rec, prec)
fpr, tpr, thr = roc_curve(y_test[:,2], predit[:,2])
auc_val = auc(fpr, tpr)
print(aupr_val,auc_val)

In [ ]:
# predit

In [ ]:
predit[:,0].shape 

In [ ]:
prec

In [ ]:
predicts = []
for a,b,c in predit:
    if a >b and a>c:
        predicts.append(0)
        continue
    if c>b and c>a:
        predicts.append(2)
        continue
    else:
        predicts.append(1)

In [ ]:
# f = (data.values[800:,2]+1)
cm = metrics.confusion_matrix(list(data.values[800:,2]+1),list(predicts))
print(cm)

CR = classification_report(list(data.values[800:,2]+1),list(predicts))
print(CR)
i=0
for j in list(data.values[800:,2]+1):
    if j==1:
        i +=1
print(i)

# plt.show()
plot_confusion_matrix_from_data(list(predicts), list(data.values[800:,2]+1))


